In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import requests
import re

# Load environment variables
load_dotenv()
client = OpenAI()

def generate_paragraphs(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"Create detailed paragraphs for a voiceover based on the following topic.I want the paragraphs only. no numbers or instruction included.: {prompt}"
            }
        ],
        max_tokens=500,
    )
    paragraphs = response.choices[0].message.content
    print("Generated paragraphs:", paragraphs)
    return paragraphs

def clean_paragraphs(paragraphs):
    # Simply return the paragraphs after stripping whitespace
    cleaned_paragraphs = [p.strip() for p in paragraphs.split('\n') if p.strip()]
    cleaned_text = ' '.join(cleaned_paragraphs)
    print("Cleaned paragraphs for voiceover:", cleaned_text)
    return cleaned_text

def generate_image_prompts(user_prompt, num_prompts=5):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": f"Create {num_prompts} distinct image prompts for visually representing: {user_prompt}"
            }
        ],
        max_tokens=100,
    )
    prompts_text = response.choices[0].message.content
    prompts = [prompt.strip() for prompt in prompts_text.split('\n') if prompt.strip()]
    
    if len(prompts) > num_prompts:
        prompts = prompts[:num_prompts]
    elif len(prompts) < num_prompts:
        prompts.extend(['Repeat for variation'] * (num_prompts - len(prompts)))

    print("Generated image prompts:", prompts)
    return prompts

def generate_image(prompt, output_filename):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url
    image_data = requests.get(image_url).content

    with open(output_filename, "wb") as f:
        f.write(image_data)
    print(f"Image saved to {output_filename}")

def generate_voiceover(text, output_filepath):
    response = client.audio.speech.create(
        model="tts-1-hd",
        voice="fable",
        input=text
    )
    response.stream_to_file(output_filepath)
    print(f"Voiceover saved to {output_filepath}")

def create_video(images, audio_path, output_video_path):
    audio = AudioFileClip(audio_path)
    audio_duration = audio.duration
    image_duration = audio_duration / len(images)

    clips = [ImageClip(img).set_duration(image_duration) for img in images]
    video_clip = concatenate_videoclips(clips, method="compose")
    video_clip = video_clip.set_audio(audio)

    fps = 24
    video_clip.write_videofile(output_video_path, codec="libx264", audio_codec="aac", fps=fps)
    print(f"Video created at {output_video_path}")

def main():
    user_prompt = input("Please describe the topic you want to create video for: ")

    # Generate the paragraphs for the video voiceover using GPT-4 based on user prompt
    paragraphs = generate_paragraphs(user_prompt)

    # Clean the paragraphs for smooth voiceover narration
    cleaned_text = clean_paragraphs(paragraphs)

    # Generate image prompts based on the user prompt
    image_prompts = generate_image_prompts(user_prompt, num_prompts=5)

    os.makedirs("./vid_images2", exist_ok=True)
    os.makedirs("./voice2", exist_ok=True)

    image_files = [f"./vid_images2/vid_image{i+1}.png" for i in range(5)]

    # Generate images based on generated prompts
    for prompt, filename in zip(image_prompts, image_files):
        generate_image(prompt, filename)

    voiceover_path = "./voice2/speech.mp3"
    generate_voiceover(cleaned_text, voiceover_path)

    create_video(image_files, voiceover_path, "output_video.mp4")

if __name__ == "__main__":
    main()


Generated paragraphs: Avocados, a world-renowned superfood, are not simply delicious but also packed with an astounding array of nutritional benefits. They are an indispensable ingredient in a healthy diet, thanks to their high-quality monounsaturated fatty acids and wide array of essential vitamins and minerals. Including this verdant fruit in your diet can be a cornerstone in the pursuit of healthy living. Let's delve deeper into the nutritional intricacies of this fruit, promising good health.

The heart of the avocado is its healthy monounsaturated fats, particularly oleic acid, which is exceptionally beneficial for heart health. These good fats help in reducing bad LDL cholesterol levels while boosting the level of good HDL cholesterol. Consequently, they significantly improve overall heart health, decreasing the risk of heart diseases. They also help your body absorb fat-soluble vitamins like A, D, E, and K more effectively. These avocados are indeed the keepers of your heart!

A

MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
Video created at output_video.mp4
